# Week 6 — Part 03: Chunking long text + synthesizing summaries

**Estimated time:** 60–100 minutes

## Learning Objectives

- Implement a simple chunking utility for long text
- Define a per-chunk summary schema
- Synthesize chunk outputs into a final summary
- Understand chunk-boundary pitfalls and overlap strategies


## Overview

When text is too long for the context window:

1. **Split into chunks**
2. **Process each chunk** into a small structured summary
3. **Synthesize** a final summary from per-chunk outputs

---

## Underlying theory: chunking is a strategy for bounded-context reasoning

If the document is longer than what you can send at once, you have two options:

- **lossy compression**: summarize first (risk losing details)
- **chunking**: split and process pieces (risk missing cross-chunk dependencies)

Chunking introduces a boundary effect: information near chunk boundaries can be separated.

Practical implication: if correctness depends on cross-paragraph links, you may need overlap or a second pass.

from __future__ import annotations


def chunk_text_simple(text: str, *, chunk_size: int = 1800) -> list[str]:
    return [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]


def chunk_text_overlap(text: str, *, chunk_size: int = 1800, overlap: int = 150) -> list[str]:
    chunks = []
    step = max(1, chunk_size - overlap)
    for i in range(0, len(text), step):
        chunks.append(text[i : i + chunk_size])
    return chunks


def summarize_chunk_stub(chunk: str) -> dict:
    return {
        "summary_bullets": [chunk[:40] + "..."],
        "key_entities": ["<todo>"] if chunk else [],
        "open_questions": [],
    }


def synthesize_summaries(summaries: list[dict]) -> dict:
    bullets = [b for s in summaries for b in s.get("summary_bullets", [])]
    return {"summary": bullets[:5], "note": "synthesized from chunk summaries"}


text = "B" * 4800
chunks = chunk_text_overlap(text, chunk_size=1800, overlap=150)
per_chunk = [summarize_chunk_stub(c) for c in chunks]
final_summary = synthesize_summaries(per_chunk)
print("chunks=", len(chunks))
print(final_summary)

## Synthesis pattern (high-level)

- For each chunk: ask the model for a short structured summary.
- After all chunks: ask the model to combine those summaries.

This is a simple form of hierarchical summarization:

- level 1: per-chunk summaries
- level 2: global synthesis

Practical implication: enforcing a small per-chunk schema helps reduce drift and makes synthesis more reliable.

In [ ]:
def chunk_text_todo(text: str, chunk_size: int = 1500, overlap: int = 200) -> list[str]:
    # TODO: implement chunking with overlap.
    raise NotImplementedError


def make_chunk_prompt(chunk: str) -> str:
    # TODO: create a prompt template for per-chunk summaries.
    raise NotImplementedError


print("Implement chunk_text_todo() and make_chunk_prompt().")